In [1]:
import numpy as np
from scipy.constants import hbar,h,e,c
from scipy.constants import elementary_charge as elc
F0 = 2.0678e-15

In [2]:
e_eff=(11.45+1)/2## for all calculations
Mhz=(2*np.pi*1e6)## divide on it to obtain value in mhz
j=0.5e-3      # critical current density nA/nm^2

In [3]:
import pandas as pd

# Q1 related parameters

In [4]:
def cap_func(Q1,Qg1,Qg2):
    return Q1+(1/Qg1+1/Qg2)**(-1)

In [5]:
def Ec(C):#fF
    return np.round((elc)**2/(2*C)/h/1e9*1e15,1) # return in MHz

In [6]:
# path=r'E:\Github2.0\QCreator\designs\ThreeQubitsPlusTwoQubits\maxwell_simulations'
path=r'C:\Python\Github\QCreator\designs\ThreeQubitsPlusTwoQubits'
# path=r'D:\NextCloud\Quantum Computing\ChipDesigns\Ivan_Gerhard_designs\Two_qubits_plus_coupler\Hamiltonian_creation'

In [46]:
data=pd.read_csv(path+'\Design_single_qubit_1.txt',sep='\t')# matrix should be done on pF

In [47]:
data

,Charge,ground,Q1_bottom,Q1_top,Resonator
Charge,2.663900e-03,-0.002622,-0.000002,-0.000039,-5.659000e-07
ground,-2.622400e-03,0.665360,-0.086433,-0.543760,-3.253700e-02
Q1_bottom,-1.689300e-06,-0.086433,0.128110,-0.038207,-3.468700e-03
Q1_top,-3.924800e-05,-0.543760,-0.038207,0.600100,-1.808700e-02
Resonator,-5.659000e-07,-0.032537,-0.003469,-0.018087,5.409300e-02


In [48]:
q_name='Q1_'
Q1_cap_ground_top=data[q_name+'top'][3]+data[q_name+'top'][2]
Q1_cap_ground_bottom=data[q_name+'bottom'][2]+data[q_name+'bottom'][3]
Q1_cap_top_bottom=-data[q_name+'bottom'][3]
Q1_cap=cap_func(Q1_cap_top_bottom,Q1_cap_ground_top,Q1_cap_ground_bottom)

In [49]:
Q1_cap_top_bottom,Q1_cap_ground_bottom,Q1_cap_ground_top

(0.038207, 0.08990300000000001, 0.561893)

In [50]:
Q1_cap

0.11570957193815243

In [51]:
Ec(Q1_cap)

167.40386296660614

In [52]:
def I(f,Ec):
    Ej=Ejf(f,Ec)
    I=Ej*2*np.pi/F0*h*1e18
    return I
def Ejf(f,Ec):
    Ej=(f+Ec)**2/(8*Ec)
    return Ej
def Eji(I):#nA
    return I*F0/(2*np.pi)/1e9/h/1e9
def IEj(Ej):
    return Ej/(F0/(Q2*np.pi)/1e9/h/1e9)
def f(Ej,Ec):
    return np.sqrt(8*Ej*Ec)-Ec
def Ec(C):#fF
    return elc**2/(2*C)/h/1e9*1e15

In [53]:
a1    = 0.300
a2    = 0.300 # Junction width in um
#Calculate Qubit frequency:
JJ_area = a1*a2*1e6 #in nm^2
print('Critical current density is: ',j*1e3, 'uA/um^2')
Ic = JJ_area*j #in nA
print('Critical current: ',Ic ,' nA')
print('Ej: ',Eji(Ic) ,' GHz')

print('junction area  in um: ',JJ_area*1e-6)

Critical current density is:  0.5 uA/um^2
Critical current:  45.0  nA
Ej:  22.35039212442379  GHz
junction area  in um:  0.09


In [54]:
f(Eji(Ic),Ec(Q1_cap)/1e3)

5.303641361081138

In [55]:
Eji(Ic)/(Ec(Q1_cap)/1e3)

133.51180628897595

### Resonator coupling

In [56]:
def line_qubit_coupling(Q1g,Q2g,Q1gc,Q2gc):
    return np.abs((Q1g*Q2gc-Q2g*Q1gc)/(Q1g+Q2gc+Q2g+Q1gc))

In [57]:
def cap_pretty_print(args,coeff):
    for i in args:
        print(np.round(i*coeff,1))# return in fF

In [58]:
Q1_cap_res_top=-data[q_name+'top'][-1]
Q1_cap_res_bottom=-data[q_name+'bottom'][-1]
Ccr=line_qubit_coupling(Q1_cap_ground_top,Q1_cap_ground_bottom,Q1_cap_res_top,Q1_cap_res_bottom)

In [62]:
cap_pretty_print([Q1_cap_res_top,Q1_cap_res_bottom,Ccr],1e3)

18.1
3.5
0.5


In [63]:
def qubit_res_coupling(Cc,Cq,wq,wr,Zr):# capacitances in fF, frequencies in GHz
    g= Cc*10**(-12)/2*np.sqrt((2*np.pi*wq*10**(9)*Zr/(Cq*10**(-12))))*wr*10**(9)
    return np.round(g/1e6,1)# return g/2pi in MHz

In [64]:
Zr=52.6
omega_r=5.4
omega_q=5.3
gqr=qubit_res_coupling(Ccr,Q1_cap,omega_q,omega_r,Zr)
gqr

5.0

### Dispersive shift

In [28]:
def dispersive_shift(gqr,wq,wr,Ec): # gqr in MHz,wq,wr in GHZ, EC in MHz and positive
    chi=gqr**2/((wq-wr)*10**(3))/(1+(wq-wr)*10**(3)/(-Ec))
    return chi # in MHz

In [29]:
chi=dispersive_shift(gqr,omega_q,omega_r,Ec(Q1_cap))
chi

-0.06030030530168379

### Readout resonator properties

In [57]:
Q=10000
k=2*np.pi*omega_r*1e9/Q 
np.round(1/k*1e9,1), np.round(k/Mhz,2)# first value in ns

(294.7, 0.54)

In [58]:
Tp=(1/k)*((omega_q-omega_r)*10**3/gqr)**2
Tp*1e6#us

120.286246748741

### Charge line

In [59]:
Q1_cap_charge_line_top=-data[q_name+'top'][0]
Q1_cap_charge_line_bottom=-data[q_name+'bottom'][0]
Cc_charge=line_qubit_coupling(Q1_cap_ground_top,Q1_cap_ground_bottom,Q1_cap_charge_line_top,Q1_cap_charge_line_bottom)
cap_pretty_print([Q1_cap_charge_line_top,Q1_cap_charge_line_bottom,Cc_charge],1e6)#in aF

76.6
93.6
10.5


In [60]:
Cc_charge*1e6

10.545244379730185

In [62]:
omega_q=4.2

In [63]:
Z0=50
Tcharge=(Cc_charge+Q1_cap)/(Z0*(2*np.pi*omega_q*Cc_charge)**2)*1e-6

In [64]:
Tcharge*1e6#in us

25026.113067607275

### Total relaxation= resonator+charge line

In [65]:
Tp*Tcharge/(Tp+Tcharge)*1e6

119.71086492265829